In [ ]:
! mkdir -p datasets
%cd datasets
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/Iris.csv
%cd ..

# Clasificación Multiclase
Cambiamos el nombre porque ¡tenemos una nueva tarea!, otra vez 😃.   
Tenemos **tres estrategias** para predecir a cuál de las especies de iris pertenece un ejemplo, aplicamos una de ellas a medias (¿o a tercias?) en el capítulo anterior, veamos cuál es.

## One vs the Rest
La idea es etiquetar **una clase como positiva** y **todas las demás como negativas**, es **una contra el resto**. En el capítulo anterior distinguimos entre **setosa** y **no-setosa**, esto lo hacemos para todas las clases, entrenando un **clasificador binario distinto** para cada caso.

Completa el siguiente código para importar los datos 😉.

In [ ]:
import pandas ___
from sklearn.model_selection import ___
import numpy as np

np.random.___

df = ___.read_csv("./datasets/Iris.csv")
y = df["___"]

# no necesitamos estas columnas
# guardamos las demás en X
X = df.drop(columns=['Id', 'Species'])

_, _, _, _ = train_test_split(X, y, test_size=0.2)

Esto no es muy complicado de implementar, pero el objeto `LogisticRegression` ya cuenta con la opción de usar dicha estrategia, estableciento su parámetro `multi_class` en `'ovr'`.  
El modelo espera que `y` sea un array 1D, por lo que **no es necesario que realicemos el one-hot encoding**.  

In [ ]:
from sklearn.linear_model import LogisticRegression

model_log = LogisticRegression(multi_class="ovr")
model_log.fit(X_train, y_train)

# realicemos algunas predicciones
X_sample = X_test.sample(5)
model_log.predict(X_sample)

También tenemos la implementación en `OneVsRestClassifier` de `sklearn.multiclass`, este recibe una **instancia del modelo** que usaremos, por supuesto, debe ser uno de **regresión logística**.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

model_ovr = OneVsRestClassifier( LogisticRegression() )
model_ovr.fit(X_train, y_train)
model_ovr.predict(X_sample)

## One vs One
Entrenamos clasificadores por pares, **uno contra uno** distinguimos setosa de versicolor, setosa de virginica, versicolor de virginica, y así hasta entrenar todas las posibles combinaciones, siendo en total:

$\dfrac{K (K-1)}{2} $

Donde $K$ es el número de clases.

La ventaja es que cada clasificador se entrena **sólo con los datos que contienen los pares de clases a distinguir**, algunos modelos **no escalan bien con el tamaño del dataset**, es decir, rinden peor con un dataset más grande.  
En estos casos, **es más rápido entrenar muchos clasificadores en pequeños datasets** en lugar de  **pocos clasificadores en datasets más grandes**. 

Usaremos `OneVsOneClassifier` también de `sklearn.multiclass` pues `LogisticRegression` no soporta esta estrategia. Completa para ver la predicción 😱.

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

model_ovo = OneVsOneClassifier(LogisticRegression())
model_ovo.fit(X_train, y_train)
model_ovo.predict(___)

## Regresión logística multinomial
Es una **generalización** de la regresión logística, que cubre **múltiples clases** directamente, eliminando la necesidad de entrenar varios clasificadores binarios.

El parámetro `multiclass` que vimos antes acepta los parámetros `'auto'`, `'ovr'` y `'multinomial'`, **auto** es la opción por defecto, y esta selecciona **ovr** si la variable `y` es **binaria**.

Como no es nuestro caso, seleccionará **multinomial**, por lo que sería opcional especificarlo.

In [ ]:
model_multi = LogisticRegression(multi_class='___')
model_multi.fit(X_train, y_train)

model_multi.predict(X_sample)

# Regresión Softmax
Esta generalización es también conocida como **regresión Softmax**. Dado un ejemplo $x$, esta calcula primero un puntaje $s_k(x)$ para cada clase $k$

$$ s_k(x) = x^T\theta^{(k)} $$

Nota que **cada clase** tiene su propio **vector** $\theta^{(k)}$. Cada uno se guarda en una fila de la matríz $\theta$ que está repartida en los atributos `coef_` e `intercept_` de nuestro modelo. A diferencia de lo que vimos al implementar el [descenso del gradiente](5_descenso_del_gradiente.ipynb), donde nuestra variable `theta` contenía los coeficientes y el término independiente en una sola matriz.

In [ ]:
 model_multi.coef_

In [ ]:
model_multi.intercept_

Una columna por cada feature, una fila por cada clase 😉

Luego se aplica la función **softmax** para calcular las probabilidades $\hat{p}_k$ de que nuestro ejemplo $x$ **pertenezca a una clase $k$** calculando una fórmula usando exponenciales, cada una de estas representan al número $e$ elevado a los puntajes $ s_k(x) $ que antes calculamos.

$\hat{p}_k = \dfrac{\exp s_k(x)}{\sum^K_{j=1}\exp s_j(x)}$

Donde $K$ es el número total de clases

In [ ]:
len(model_multi.classes_)

Al final, la predicción toma en cuenta la clase con **la probabilidad más alta como respuesta**

## Implementación de sus fórmulas
Primero calculemos $s(x)$ como un vector que contega los **puntajes de cada clase**, por lo que tendrá 3 elementos al ser el resultado de una multiplicación de matrices, y no olvidemos sumar los términos independientes 😉.

In [ ]:
# Operamos con una matriz de numpy
# y guardamos la primera fila
X_sample1 = X_sample.to_numpy()[0]

s = X_sample1 @ model_multi.coef_.T + model_multi.intercept_
s

Luego $p$ como otro vector de tres elementos que contiene las **probabilidades** de que nuestro ejemplo **pertenezca a alguna de las clases** 0, 1 o 2, setosa, versicolor o virginica, respectivamente 😄

In [ ]:
p = np.exp(s) / np.sum(np.exp(s))
p

Finalmente obtenemos a qué clase pertenece nuestro ejemplo 😃.  
Podemos usar `np.argmax` sobre el **vector que conteniene las probabilidades**, y esto nos retorna el **índice donde se encuentra la probabilidad más alta**.

In [ ]:
# Clase de nuestro primer ejemplo
pred = np.argmax(p)
pred

Es **versicolor**, como nuestros anteriores modelos lo predijeron 😄

# Ejercicios
Intenta predecir **todo** `X_sample`, usando esta implementación al final `pred` debería ser un **array de 5 elementos** con cada una de las clases predichas para cada ejemplo 😉.    
Además, `s` y `p` deberían ser matrices de 3x5 

In [ ]:
# reemplaza X_sample1


¿Alguna estrategia rendirá mejor sobre las otras?
Comprúebalo evaluando los 3 modelos sobre precision y recall

In [ ]:
# importa las métricas, evalúa siempre sobre el test set


Cada estrategia tiene su momento: **OvR** si el modelo necesitamos algo **estrictamente binario**, **OvO**  si el modelo **no escala bien con la cantidad de datos** y la **regresión softmax** es una regresión logística generalizada 😃.

Las primeras dos nos dicen que existen **más tipos de modelos de clasificación**, pero el próximo que veremos usa softmax al final 😂 pero no por ello es menos interesante, es más, te aseguro que su nombre te despierta interés 🧠:

Te presento a las [redes neuronales artificiales](10_Redes_neuronales.ipynb) 